# 데이터 환경 설정

In [ ]:
!conda install -c conda-forge shapely
!conda install -c conda-forge pyproj
!conda install -c conda-forge haversine
!pip install wikipedia-API

In [1]:
import folium
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import decimal
from shapely import wkt
from pyproj import Proj, transform
from haversine import haversine

## 데이터 불러오기

In [2]:
df_ins = pd.read_csv('./data/202011_자연환경조사_곤충_점.csv')
df_rep = pd.read_csv('./data/202011_자연환경조사_양서파충류_점.csv')
df_spi = pd.read_csv('./data/202011_자연환경조사_저서무척추동물_점.csv')
df_bird = pd.read_csv('./data/202011_자연환경조사_조류_점.csv')
df_mam = pd.read_csv('./data/202011_자연환경조사_포유류_점.csv')

In [3]:
df_ins['lati'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_ins['long'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_rep['lati'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_rep['long'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_spi['lati'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_spi['long'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_bird['lati'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_bird['long'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_mam['lati'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_mam['long'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

In [4]:
df_all = pd.concat([df_ins, df_rep, df_spi, df_bird, df_mam]).copy()
df_all = df_all.reset_index(drop=True)

In [5]:
inProj = Proj(init='epsg:5186')
outProj = Proj(init='epsg:4326')

kr_long, kr_lati = transform(inProj,outProj, list(df_all['lati']), list(df_all['long']))

df_all['kr_lati'] = kr_lati
df_all['kr_long'] = kr_long

In [6]:
user = (35.233512, 129.0722714)
#user = list(map(decimal.Decimal, input().split(',')))

35.233512, 129.0722714


In [7]:
df_want = df_all[(user[0]-decimal.Decimal(0.5) < df_all['kr_lati']) & (user[0]+decimal.Decimal(0.5) > df_all['kr_lati'])& (user[1]+decimal.Decimal(0.5) > df_all['kr_long'])& (user[1]+decimal.Decimal(0.5) > df_all['kr_long'])].reset_index(drop=True)

In [8]:
want_list = []
for i in df_want.index:
    if haversine(user, (df_want['kr_lati'][i], df_want['kr_long'][i]), unit = 'km') <= 3:
        want_list.append(i)
        
df_find = df_want.loc[want_list].copy()

In [112]:
import wikipediaapi
import requests
import json

def get_wiki_main_image(title):
    url = 'https://ko.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : title
    }
    response = requests.get(url, data, verify=False)
    json_data = json.loads(response.text)
    return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'

wiki=wikipediaapi.Wikipedia('ko')
wiki_dic = {}
img_dic = {}

In [118]:
zoo_map = folium.Map(location=[user[0], user[1]], zoom_start = 14)
marker_dic = {'곤충' : 'yellow',
              '양서파충류' : 'blue',
              '저서무척추동물' : 'red',
              '조류' : 'ivory', 
              '포유류' : 'green'}
tooltip = '나를 눌러줘!'

for i in want_list:
    # page dic 생성
    if df_want.iloc[i]['spcs_lcnm'] not in wiki_dic.values() :
        page_wiki = wiki.page(df_want.iloc[i]['spcs_lcnm'])
        wiki_dic[df_want.iloc[i]['spcs_lcnm']] = page_wiki
    else :
        page_wiki = wiki_dic[df_want.iloc[i]['spcs_lcnm']]
    
    # img dic 생성
    if page_wiki.exists() == True :
        if (page_wiki.title not in img_dic.values()) :
            wiki_img = get_wiki_main_image(page_wiki.title)
            img_dic[page_wiki.title] = wiki_img
        else : wiki_img[page_wiki.title]
    else :
        wiki_img = "./data/고라니.jpg"
                
    # 팝업창 요약설명 설정
    if page_wiki.exists() == True : summary = page_wiki.summary.split('.')[0]
    else : summary = df_want.iloc[i]['spcs_lcnm'] + '은(는) ' + df_want.iloc[i]['examin_realm_se_code_nm'] + '이다'
    
    # 팝업창 제목 추가
    popup = '<h3>나는 <strong>' + df_want.iloc[i]['spcs_lcnm'] + '</strong>야!</h3>'
    # 팝업창 이미지 추가
    popup += '<img src=' + wiki_img + ' class="Popup-img">'
    # 팝업창 요약설명 추가
    popup += '<p><h4>' + summary + '.</h4>'
    if page_wiki.exists() == True : popup += '<b><a href=' + page_wiki.canonicalurl + '>더 알아보기</a></b></p>'
    
    folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], popup=popup, 
                        tooltip=tooltip, radius=4, color=marker_dic[df_want.iloc[i]['examin_realm_se_code_nm']]).add_to(zoo_map)

zoo_map.save('map.html')
zoo_map

In [120]:
len(want_list)

341